<a href="https://colab.research.google.com/github/satomko/pdfchatbot/blob/main/pdfChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -U langchain langchain_community langchain-openai openai faiss-cpu tiktoken
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.106.1
    Uninstalling openai-1.106.1:
      Successfully uninstalled openai-1.106.1


In [39]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [40]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import files

# Upload the PDF file
uploaded = files.upload()

# Get the name of the uploaded file
file_name = list(uploaded.keys())[0]

# Load the PDF
loader = PyPDFLoader(file_name)
pages = loader.load_and_split()

# Chunk it up
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=24)
chunks = text_splitter.split_documents(pages)

Saving Scott_Tomko_Resume.pdf to Scott_Tomko_Resume (6).pdf


In [41]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(chunks, embeddings)

In [42]:
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from google.colab import userdata

qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(),
    db.as_retriever(),
    return_source_documents=True
)

chat_history = []

def chat(query):
    result = qa_chain({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))
    return result['answer']

In [43]:
print(chat("What's the main topic of this PDF?"))
print(chat("Can you summarize the key points?"))

/tmp/ipython-input-3583193290.py:14: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": query, "chat_history": chat_history})


The main topic of the provided context is around the achievements and leadership impact of a Senior Manager in Software Engineering at Adobe. It covers areas such as problem-solving, critical thinking, stakeholder relationship management, resource and budget planning, talent development, performance management, cross-functional leadership, incident management, OKRs, KPIs, and the transformation of monolithic-to-microservices redesign into a cloud-native architecture at Adobe.
The Senior Manager at Adobe led globally distributed software engineering teams focused on cloud-native Java microservices for analytics and workflow orchestration, contributing to enhancing customer satisfaction and enabling new machine learning capabilities. They built and scaled high-performing engineering teams, drove technical transformation, developed and patented multiple machine learning features, and fostered a culture of innovation that strengthened Adobe Analytics' competitive positioning.


In [44]:
print(chat("What is the most recent company where the candidate has worked?"))

The most recent company where the candidate has worked is Adobe in Lehi, UT, where they have been employed since November 2010.
